### Jul AdventKalender D16

https://adventofcode.com/2024/day/16

#### Part 1

In [1]:
import numpy as np
np.set_printoptions(linewidth=np.inf)

def read_file():
    input_file_path = "data/input16.txt"
    map_maze = []
    def process_line(line, if_upper):
        map_maze.append(list(line.strip()))

    with open(input_file_path, 'r') as file:
        for line in file:
            process_line(line, True)            
    return np.array(map_maze)

map_maze = read_file()
start_pos = tuple(i[0] for i in np.where(map_maze=="S"))
end_pos = tuple(i[0] for i in np.where(map_maze=="E"))

In [2]:
import heapq

rows, cols = map_maze.shape

actions = [0, 1, -1] # move, turn right, turn left

move = [(0, 1), (1, 0), (0, -1), (-1, 0)] # [move right, down, left, up]
faces = [0, 1, 2, 3] # facing directions

def is_valid_move(x, y):
#     global rows, cols
    return 0 <= x < rows and 0 <= y < cols and map_maze[x, y] != "#"

def is_found_goal(x, y):
    return x == end_pos[0] and y == end_pos[1]

def Dijkstra_find_path():
    visited = set() # pos=(x, y), (face, action_next)
    predecessors = {}  # to track paths: (pos, face, action_next) -> [(previous_pos, previous_face, previous_action)]

    todolist = []
    for action in actions:
        todolist.append((0, start_pos, (faces[0], action))) # todolist: cost, pos=(x, y), face, action_next
    
    while(todolist):
        cost, pos, (face, action_next) = heapq.heappop(todolist) # pop the smallest item(smallest cost)
        if (pos, (face, action_next)) in visited:
            continue
        visited.add((pos, (face, action_next)))
        
        if is_found_goal(pos[0], pos[1]):
            return cost, visited, predecessors
            
        if_actioned = False
        if action_next != 0:
            nface = (face + action_next)%len(faces)
            nx, ny = pos
            
            new_cost = cost + 1000 # for turning
            if_actioned = True
        else:
            nface = face
            nx, ny = pos[0] + move[face][0], pos[1] + move[face][1]
            
            if is_valid_move(nx, ny):
                new_cost = cost + 1 # for moving forward
                if_actioned = True
                
        if if_actioned: # otherwise deadend
            for action_prepare in actions: # for todolist
                if ((action_prepare == 0 and action_next == 0 and 
                     move[nface][0] == -move[face][0] and move[nface][1] == -move[face][1]) 
                    or (action_prepare != 0 and action_next != 0 and nface == -face)): # not go back
                    continue
                heapq.heappush(todolist, (new_cost, (nx, ny), (nface, action_prepare)))
                
                # add predecessors
                if ((nx, ny), nface, action_prepare, new_cost) not in predecessors:
                    predecessors[((nx, ny), nface, action_prepare, new_cost)] = []
                predecessors[((nx, ny), nface, action_prepare, new_cost)].append((pos, face, action_next, cost))
    return None, visited, predecessors

In [3]:
cost, visited, predecessors = Dijkstra_find_path()
print(cost)

102460


#### Part 2

In [4]:
def reconstruct_paths(optimal_cost):
#     paths = []
    positions = set()
    todolist = []
    for face in faces:
        px, py = end_pos[0] - move[face][0], end_pos[1] - move[face][1]
        if is_valid_move(px, py):
            todolist.append([(end_pos, face, 0, optimal_cost)]) # todolist: paths, each path is a state: pos=(x, y), face, action, cost

    def if_start(state):
        for action in actions:
            if state == (start_pos, faces[0], action, 0):
                return True
        return False

#     print(todolist)
    while todolist:
#         print(todolist)
        path = todolist.pop()
        current = path[-1]
        
        if if_start(current):
            for state in path:
                positions.add(state[0])
#             paths.append(path[::-1])  # Reverse path for correct order
#             print(paths)
            continue

        for parent in predecessors[current]:
            if parent not in path and parent[-1] < current[-1]:
                todolist.append(path + [parent])
    return len(positions) # paths

# optimal_paths = reconstruct_paths(cost)
reconstruct_paths(cost)

527

In [5]:
# positions = set()
# for path in optimal_paths:
#     for state in path:
#         positions.add(state[0])
# len(positions)